# Import the necessary packages 

In [1]:
import pandas as pd
import numpy as np
#import pandas_datareader.data as web
from datetime import date, datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm


# Load data sets

In [2]:
#Load S&P 500 data
spx = pd.read_csv('..\data\$spx_intraday-30min.csv')
#Split Time into data and Time columns
spx['date'] = [datetime.strftime(datetime.strptime(x.split(' ')[0], '%m/%d/%Y'),'%Y-%m-%d')for x in spx['Time']]
spx['Time'] = [x.split(' ')[1] for x in spx['Time']]

#Calculate log Returns over 30 min increments
spx['Return'] = np.log(spx['Last'])- np.log(spx['Open'])

#Load lstm balanced data set
lstm_bal = pd.read_csv('C:/Users/Elise Nguyen/Applied DS/Fall2020-Project5-fall2020-project5-group-2/output/results/LSTM_balanced_results.csv',index_col=0)
#lstm_bal = lstm_bal.rename(columns={0 :'Column 0'},inplace=True)

#Rounding time to the closest half hour
def round_time(time):
    if int(time.split(':')[1])<30:
        return str(time.split(':')[0]+':00')
    else:
        return str(time.split(':')[0]+':30')

lstm_bal['Time'] = [round_time(x) for x in lstm_bal['time']]

#Merging S&P data with lstm_bal data sets by data and Time columns
spx_reg = spx.merge(lstm_bal, how = 'outer', on = ['date','Time'])
#Drop unnecessary columns
spx_reg = spx_reg.drop(['id','content','High','Low','Volume','Change','negative_prob','positive_prob'],axis = 1)
spx_reg['outcome'] = spx_reg['outcome'].fillna(0)
print(spx_reg.head(20))

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Elise Nguyen/Applied DS/Fall2020-Project5-fall2020-project5-group-2/output/results/LSTM_balanced_results.csv'

In [ ]:
#Separate spx_reg data set into Neutral(no tweets), Positive (positive tweets), & negative (negative tweets)
spx_neutral = spx_reg[spx_reg['outcome']==0]
spx_pos = spx_reg[spx_reg['outcome']==1]
spx_neg = spx_reg[spx_reg['outcome']==-1]
print(spx_pos.shape)
print(spx_neg.head(5))
print(spx_pos.head(5))



In [ ]:
#Drop rows of 30 min with conflicting sentiment tweets (both + and - in the same 30 min)
neg_drop_index = []
pos_drop_index = []
for i in range(0,spx_neg.shape[0]):
    for j in range(0,spx_pos.shape[0]):
        if spx_neg.iloc[i,0]== spx_pos.iloc[j,0] and spx_neg.iloc[i,3]== spx_pos.iloc[j,3]:
            neg_drop_index.append(i)
            pos_drop_index.append(j)

spx_neg2 = spx_neg.drop(spx_neg.index[neg_drop_index])
spx_pos2 = spx_pos.drop(spx_pos.index[pos_drop_index])

In [ ]:
print(spx_reg.head(20))
print(spx_pos2.head(10))
print(spx_neg2.head(10))


In [ ]:
#Concatenating postive and neutral tweets 
spx_pos_reg = pd.concat([pd.DataFrame(spx_pos2), pd.DataFrame(spx_neutral)])

#Try running regression for Time = 10.30 only
spx_pos_reg = spx_pos_reg[spx_pos_reg['Time']=='10:00']
# print(spx_pos_reg['Return'])
# print(spx_pos_reg['outcome'])
plt.scatter(spx_pos_reg['outcome'],spx_pos_reg['Return'])
pos_model = sm.OLS(spx_pos_reg['Return'],spx_pos_reg['outcome']).fit()
pos_model.summary()

In [ ]:
spx_pos_neg

In [ ]:
#Concatenating postive and negative tweets 
spx_pos_neg = pd.concat([pd.DataFrame(spx_pos2), pd.DataFrame(spx_neg2)])

#Try running regression for Time = 10.30 only
spx_pos_neg = spx_pos_neg[spx_pos_neg['Time']=='10:00']
# print(spx_pos_reg['Return'])
# print(spx_pos_reg['outcome'])
plt.scatter(spx_pos_neg['outcome'],spx_pos_neg['Return'])
plt.xlabel('Sentiment'), plt.ylabel('Return'),plt.title('Tweet Sentiment vs. S&P Returns')
model = sm.OLS(spx_pos_neg['Return'],spx_pos_neg['outcome']).fit()
model.summary()